# Movie_Recommendation

Objective: To create a movie recommendation system based on movie features like genre, keywords, tagline, cast, and director. The code uses the concept of TF-IDF (Term Frequency-Inverse Document Frequency) vectorization to convert the movie features into numerical tokens and then calculates the cosine similarity between movies to find similar recommendations.

Data Source:
Github, YBI

**Import Library**



In [26]:
import pandas as pd

In [27]:
import numpy as np

**Import Dataset**

In [28]:
df = pd.read_csv('https://raw.githubusercontent.com/YBI-Foundation/Dataset/main/Movies%20Recommendation.csv')

In [29]:
df.head()

,Movie_ID,Movie_Title,Movie_Genre,Movie_Language,Movie_Budget,Movie_Popularity,Movie_Release_Date,Movie_Revenue,Movie_Runtime,Movie_Vote,...,Movie_Homepage,Movie_Keywords,Movie_Overview,Movie_Production_House,Movie_Production_Country,Movie_Spoken_Language,Movie_Tagline,Movie_Cast,Movie_Crew,Movie_Director
0,1,Four Rooms,Crime Comedy,en,4000000,22.876230,09-12-1995,4300000,98.0,6.5,...,NaN,hotel new year's eve witch bet hotel room,It's Ted the Bellhop's first night on the job....,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Twelve outrageous guests. Four scandalous requ...,Tim Roth Antonio Banderas Jennifer Beals Madon...,"[{'name': 'Allison Anders', 'gender': 1, 'depa...",Allison Anders
1,2,Star Wars,Adventure Action Science Fiction,en,11000000,126.393695,25-05-1977,775398007,121.0,8.1,...,http://www.starwars.com/films/star-wars-episod...,android galaxy hermit death star lightsaber,Princess Leia is captured and held hostage by ...,"[{""name"": ""Lucasfilm"", ""id"": 1}, {""name"": ""Twe...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","A long time ago in a galaxy far, far away...",Mark Hamill Harrison Ford Carrie Fisher Peter ...,"[{'name': 'George Lucas', 'gender': 2, 'depart...",George Lucas
2,3,Finding Nemo,Animation Family,en,94000000,85.688789,30-05-2003,940335536,100.0,7.6,...,http://movies.disney.com/finding-nemo,father son relationship harbor underwater fish...,"Nemo, an adventurous young clownfish, is unexp...","[{""name"": ""Pixar Animation Studios"", ""id"": 3}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","There are 3.7 trillion fish in the ocean, they...",Albert Brooks Ellen DeGeneres Alexander Gould ...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
3,4,Forrest Gump,Comedy Drama Romance,en,55000000,138.133331,06-07-1994,677945399,142.0,8.2,...,NaN,vietnam veteran hippie mentally disabled runni...,A man with a low IQ has accomplished great thi...,"[{""name"": ""Paramount Pictures"", ""id"": 4}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","The world will never be the same, once you've ...",Tom Hanks Robin Wright Gary Sinise Mykelti Wil...,"[{'name': 'Alan Silvestri', 'gender': 2, 'depa...",Robert Zemeckis
4,5,American Beauty,Drama,en,15000000,80.878605,15-09-1999,356296601,122.0,7.9,...,http://www.dreamworks.com/ab/,male nudity female nudity adultery midlife cri...,"Lester Burnham, a depressed suburban father in...","[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Look closer.,Kevin Spacey Annette Bening Thora Birch Wes Be...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4760 entries, 0 to 4759
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Movie_ID                  4760 non-null   int64  
 1   Movie_Title               4760 non-null   object 
 2   Movie_Genre               4760 non-null   object 
 3   Movie_Language            4760 non-null   object 
 4   Movie_Budget              4760 non-null   int64  
 5   Movie_Popularity          4760 non-null   float64
 6   Movie_Release_Date        4760 non-null   object 
 7   Movie_Revenue             4760 non-null   int64  
 8   Movie_Runtime             4758 non-null   float64
 9   Movie_Vote                4760 non-null   float64
 10  Movie_Vote_Count          4760 non-null   int64  
 11  Movie_Homepage            1699 non-null   object 
 12  Movie_Keywords            4373 non-null   object 
 13  Movie_Overview            4757 non-null   object 
 14  Movie_Pr

In [31]:
df.shape

(4760, 21)

In [32]:
df.columns

Index(['Movie_ID', 'Movie_Title', 'Movie_Genre', 'Movie_Language',
       'Movie_Budget', 'Movie_Popularity', 'Movie_Release_Date',
       'Movie_Revenue', 'Movie_Runtime', 'Movie_Vote', 'Movie_Vote_Count',
       'Movie_Homepage', 'Movie_Keywords', 'Movie_Overview',
       'Movie_Production_House', 'Movie_Production_Country',
       'Movie_Spoken_Language', 'Movie_Tagline', 'Movie_Cast', 'Movie_Crew',
       'Movie_Director'],
      dtype='object')

**Get Feature Selection**

In [33]:
df_features = df[['Movie_Genre','Movie_Keywords','Movie_Tagline','Movie_Cast','Movie_Director']].fillna('')

In [34]:
df_features.shape

(4760, 5)

In [35]:
df_features

,Movie_Genre,Movie_Keywords,Movie_Tagline,Movie_Cast,Movie_Director
0,Crime Comedy,hotel new year's eve witch bet hotel room,Twelve outrageous guests. Four scandalous requ...,Tim Roth Antonio Banderas Jennifer Beals Madon...,Allison Anders
1,Adventure Action Science Fiction,android galaxy hermit death star lightsaber,"A long time ago in a galaxy far, far away...",Mark Hamill Harrison Ford Carrie Fisher Peter ...,George Lucas
2,Animation Family,father son relationship harbor underwater fish...,"There are 3.7 trillion fish in the ocean, they...",Albert Brooks Ellen DeGeneres Alexander Gould ...,Andrew Stanton
3,Comedy Drama Romance,vietnam veteran hippie mentally disabled runni...,"The world will never be the same, once you've ...",Tom Hanks Robin Wright Gary Sinise Mykelti Wil...,Robert Zemeckis
4,Drama,male nudity female nudity adultery midlife cri...,Look closer.,Kevin Spacey Annette Bening Thora Birch Wes Be...,Sam Mendes
...,...,...,...,...,...
4755,Horror,,The hot spot where Satan's waitin'.,Lisa Hart Carroll Michael Des Barres Paul Drak...,Pece Dingo
4756,Comedy Family Drama,,It’s better to stand out than to fit in.,Roni Akurati Brighton Sharbino Jason Lee Anjul...,Frank Lotito
4757,Thriller Drama,christian film sex trafficking,She never knew it could happen to her...,Nicole Smolen Kim Baldwin Ariana Stephens Brys...,Jaco Booyens
4758,Family,,,,


In [36]:
x = df_features['Movie_Genre'] + ' ' + df_features['Movie_Keywords'] + ' ' + df_features['Movie_Tagline'] + ' ' + df_features['Movie_Cast']+ ' ' + df_features['Movie_Director']

In [37]:
x.shape

(4760,)

**Get Features Text Conversion to Tokens**

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
tfidf = TfidfVectorizer()

In [40]:
x = tfidf.fit_transform(x)

In [41]:
x.shape

(4760, 17258)

In [42]:
print(x)

  (0, 617)	0.1633382144407513
  (0, 492)	0.1432591540388685
  (0, 15413)	0.1465525095337543
  (0, 9675)	0.14226057295252661
  (0, 9465)	0.1659841367820977
  (0, 1390)	0.16898383612799558
  (0, 7825)	0.09799561597509843
  (0, 1214)	0.13865857545144072
  (0, 729)	0.13415063359531618
  (0, 13093)	0.1432591540388685
  (0, 15355)	0.10477815972666779
  (0, 9048)	0.0866842116160778
  (0, 11161)	0.06250380151644369
  (0, 16773)	0.17654247479915475
  (0, 5612)	0.08603537588547631
  (0, 16735)	0.10690083751525419
  (0, 7904)	0.13348000542112332
  (0, 15219)	0.09800472886453934
  (0, 11242)	0.07277788238484746
  (0, 3878)	0.11998399582562203
  (0, 5499)	0.11454057510303811
  (0, 7071)	0.19822417598406614
  (0, 7454)	0.14745635785412262
  (0, 1495)	0.19712637387361423
  (0, 9206)	0.15186283580984414
  :	:
  (4757, 5455)	0.12491480594769522
  (4757, 2967)	0.16273475835631626
  (4757, 8464)	0.23522565554066333
  (4757, 6938)	0.17088173678136628
  (4757, 8379)	0.17480603856721913
  (4757, 15303)	0.07

**Get Similarity Score Using Cosine Similarity**

In [43]:
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
Similarity_Score = cosine_similarity(x)

In [45]:
Similarity_Score

array([[1.        , 0.01351235, 0.03570468, ..., 0.        , 0.        ,
        0.        ],
       [0.01351235, 1.        , 0.00806674, ..., 0.        , 0.        ,
        0.        ],
       [0.03570468, 0.00806674, 1.        , ..., 0.        , 0.08014876,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.08014876, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [46]:
Similarity_Score.shape

(4760, 4760)

**Get Movie Name as Input form User & Validate for Closest Spelling**

In [52]:
Favourite_Movie_Name = input(' Enter your favourite movie name :')

 Enter your favourite movie name :love


In [53]:
All_Movies_Title_List = df['Movie_Title'].tolist()

In [54]:
import difflib

In [55]:
Movie_Recommendation = difflib.get_close_matches(Favourite_Movie_Name, All_Movies_Title_List)
print(Movie_Recommendation)

['Beloved', 'Chloe', 'Alive']


In [56]:
Close_Match = Movie_Recommendation[0]
print(Close_Match)

Beloved


In [ ]:
Index_of_Close_Match_Movie = df[df.Movie_Title == Close_Match]['Movie_ID'].values[0]
print(Index_of_Close_Match_Movie)

In [58]:
Recommendation_Score = list(enumerate(Similarity_Score[Index_of_Close_Match_Movie]))
print(Recommendation_Score)

[(0, 0.007715071890311533), (1, 0.0), (2, 0.00460582403803784), (3, 0.0171208501197933), (4, 0.0035576881444419144), (5, 0.003381876597953319), (6, 0.0), (7, 0.013990279754859072), (8, 0.024139333604827017), (9, 0.004967385756887638), (10, 0.005720339952020939), (11, 0.028575951338850464), (12, 0.009332901572397943), (13, 0.0), (14, 0.004911842244312453), (15, 0.0217907542011867), (16, 0.002810390065620565), (17, 0.0), (18, 0.008380338934332198), (19, 0.003253239077482603), (20, 0.003526831196926696), (21, 0.06441753280316576), (22, 0.0), (23, 0.003513749119253273), (24, 0.0022022421226389802), (25, 0.007229424638522223), (26, 0.003299172017586711), (27, 0.0), (28, 0.04404231463803187), (29, 0.008322648177314795), (30, 0.0), (31, 0.061820502009099755), (32, 0.0029569052637916037), (33, 0.007723616233196436), (34, 0.003508636273261681), (35, 0.025054497429463286), (36, 0.0), (37, 0.024787249099754206), (38, 0.0187615580298083), (39, 0.010779664773960739), (40, 0.033545845554649095), (41

In [59]:
len(Recommendation_Score)

4760

**Get All Movies Sort Based on Recommendation Score wrt Favourite Movie**

In [60]:
Sorted_Similar_Movies = sorted(Recommendation_Score, key = lambda x:x[1], reverse = True)
print(Sorted_Similar_Movies)

[(3230, 1.0000000000000002), (2176, 0.20610949741430645), (1254, 0.17943126857478017), (4322, 0.17332584049808605), (4363, 0.14782388569794408), (3853, 0.14507692193003602), (1263, 0.13339744051559901), (4658, 0.13144208295416168), (757, 0.1216618554129443), (595, 0.11782766678170883), (1349, 0.11752928386927713), (2730, 0.1164154953645882), (650, 0.11638858120540604), (1220, 0.11128032096043411), (4113, 0.1109416355530687), (4372, 0.10802985998995926), (2009, 0.10586234618724003), (680, 0.1043501870203017), (3041, 0.10429557915067555), (1547, 0.10298721886185737), (2309, 0.10160325309399829), (3562, 0.10110515508704167), (1837, 0.09862082279006795), (425, 0.09738937521904494), (388, 0.09727204724341365), (3388, 0.09592624016211412), (4633, 0.09414799931230176), (2683, 0.09248982823199255), (1971, 0.09203612996731093), (4507, 0.09201641513926666), (4418, 0.09173202385831902), (4382, 0.09172108899201435), (850, 0.09169450281310774), (2592, 0.09045169201600393), (1234, 0.0896938910801918

In [63]:
print('Top 20 Movies Suggested for You : \n')

i = 1
for movie in Sorted_Similar_Movies:
  index = movie[0]
  title_from_index = df[df.index==index]['Movie_Title'].values[0]
  if (i<21):
    print(i, '.',title_from_index)
    i+=1

Top 20 Movies Suggested for You : 

1 . Secretariat
2 . Dreamer: Inspired By a True Story
3 . The Man in the Iron Mask
4 . Slow West
5 . Heaven is for Real
6 . Sinister
7 . The Mask of Zorro
8 . The Infiltrator
9 . Adaptation.
10 . Red Dawn
11 . The Right Stuff
12 . Hardball
13 . Hidalgo
14 . Mary Reilly
15 . Saving Mr. Banks
16 . Dwegons
17 . Must Love Dogs
18 . The Perfect Storm
19 . Chairman of the Board
20 . The Messenger: The Story of Joan of Arc


**Top 10 Movie Recommendation System**

In [64]:
Movie_Name = input('Enter Your Favourite Movie Name: ')

list_of_all_titles = df['Movie_Title'].tolist()

Find_Close_Match = difflib.get_close_matches(Movie_Name, list_of_all_titles)

Close_Match = Find_Close_Match[0]

Index_of_Movie = df[df.Movie_Title == Close_Match]['Movie_ID'].values[0]

Recommendation_Score = list(enumerate(Similarity_Score[Index_of_Movie]))

sorted_similar_movies = sorted(Recommendation_Score, key = lambda x:x[1], reverse = True)

print('Top 10 Movies Suggested For You : \n')

i = 1
for movie in Sorted_Similar_Movies:
  index = movie[0]
  title_from_index = df[df.Movie_ID==index]['Movie_Title'].values
  if (i<11):
    print(i, '.',title_from_index)
    i+=1

Enter Your Favourite Movie Name: Mission: Impossible
Top 10 Movies Suggested For You : 

1 . ['Beloved']
2 . ['No End in Sight']
3 . ['Mortal Kombat']
4 . ['Of Horses and Men']
5 . ['The Book of Life']
6 . ['Lockout']
7 . ['The Core']
8 . ['The Boy']
9 . ['The Abyss']
10 . ['Fear and Loathing in Las Vegas']


Explaination:
The code imports the necessary libraries for data manipulation and machine learning, including pandas, numpy, and sklearn.
The movie dataset is loaded into a pandas DataFrame (df) from the provided URL using pd.read_csv().
The movie features ('Movie_Genre', 'Movie_Keywords', 'Movie_Tagline', 'Movie_Cast', 'Movie_Director') are selected and stored in a new DataFrame df_features. Missing values in these features are filled with empty strings.
The movie features are combined into a single string for each movie in the 'x' variable, which will be used for TF-IDF vectorization.
TF-IDF vectorization is applied to convert the combined movie features into numerical tokens using TfidfVectorizer from sklearn's feature_extraction.text module. The fit_transform() method is applied to 'x' to create the feature matrix x.
Cosine similarity is calculated between movies using cosine_similarity from sklearn's metrics.pairwise module, based on the TF-IDF vectors. The resulting Similarity_Score matrix represents the similarity between all pairs of movies.
The user is asked to enter their favorite movie name, and the code validates for the closest spelling match using the difflib.get_close_matches() function.
The movie recommendations are then generated based on similarity scores for the user's favorite movie. The top 20 movies with the highest recommendation scores are displayed as suggestions to the user.